# Práctica ETL – Grupo 9 (Semana 2)
**ProyectoG9.ipynb**
**Fecha:** 07/05/2025

---
## 0. Instalación de dependencias

Antes de empezar, instalamos las librerías necesarias en el entorno de Jupyter:
- `pandas` y `numpy` para manipulación y cálculo de datos.
- `sqlalchemy` y `psycopg2` para conexión a PostgreSQL (aunque en esta fase leeremos solo CSV).
- `matplotlib` para visualización rápida si fuera necesario.

In [4]:
!pip install pandas numpy sqlalchemy psycopg2-binary matplotlib



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Introducción

En esta segunda semana nos centraremos en la etapa de **Transformación** del proceso ETL:
- Exploraremos y analizaremos en detalle las tres fuentes de datos.
- Crearemos una función genérica de limpieza.
- Eliminaremos nulos en columnas clave y aplicaremos transformaciones con `lambda`.
- Expanderemos los DataFrames con nuevas columnas limpias.
- Generaremos índices numéricos únicos y secuenciales para cada registro.

---

## 2. Preparación del entorno

In [5]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot as plt


Leemos los tres archivos CSV directamente del directorio `data/`, sin interactuar con la base de datos por ahora:
1. **IoT-Malware** (`IoT-Malware.csv`)
2. **NetCrime** (`NetCrime.csv`)
3. **RansomAttacks** (`RansomAttacks.csv`)

In [6]:
# 2.1 IoT-Malware (separador pipe)
df_iot = pd.read_csv('data/IoT-Malware.csv', sep='|', engine='python')

# 2.2 NetCrime
df_net = pd.read_csv('data/NetCrime.csv')

# 2.3 RansomAttacks
df_ransom = pd.read_csv('data/RansomAttacks.csv')

# 2.4 Comprobamos dimensiones
print("IoT-Malware:", df_iot.shape)
print("NetCrime:   ", df_net.shape)
print("RansomAttacks:", df_ransom.shape)


IoT-Malware: (237, 23)
NetCrime:    (117, 13)
RansomAttacks: (358, 29)


> Con esto ya tenemos nuestras tres fuentes de datos cargadas en DataFrames para proceder a limpieza y transformación.


## 3. Carga y exploración inicial de datos
____

In [7]:
display(df_iot.head(3))
display(df_net.head(3))
display(df_ransom.head(3))

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
0,1.547151e+09,CzsY0D4B96NTr8m7ld,192.168.1.199,59222,46.101.251.172,80,tcp,http,1.686784,149,...,-,11584,ShADadttfF,122,7741,122,178102,-,Malicious C&C,FileDownload
1,1.547151e+09,CGEJbl3RNkmXzmkEd4,192.168.1.199,59224,46.101.251.172,80,tcp,-,3.081233,0,...,-,0,S,3,180,0,0,-,Malicious C&C,NaN
2,1.547151e+09,CVMYDw4wnZfqWrOfd,192.168.1.199,59224,46.101.251.172,80,tcp,-,-,-,...,-,0,S,1,60,0,0,-,Malicious C&C,NaN


,Country,2019_Complaints,2019_Losses,2020_Complaints,2020_Losses,2021_Complaints,2021_Losses,2022_Complaints,2022_Losses,2023_Complaints,2023_Losses,2024_Complaints,2024_Losses
0,PR,655,5929974,1338,7209755,1785,9463928,1594,17211758,1817,21018302,1974,31545772
1,PS,1784,22483591,2890,25423219,3352,48918464,3210,57796003,3378,69292891,3811,66002407
2,PT,1119,13870074,2020,12391290,2102,18205913,1918,30859319,2178,28700418,2209,40192274


,Target,AKA,description,sector,"organisation size 1,5,10,25,100,300",revenue $USD million,cost,ransom cost,data note,ransom paid,...,revenue as of,no of employees,Data Note,Source Name,URL,URL 2,URL 3,URL 4,URL 5,#ID
0,Kaseya,NaN,NaN,tech,300,300.0,70,70.0,NaN,unknown,...,NaN,NaN,NaN,NaN,https://techcrunch.com/2021/07/05/kaseya-hack-...,NaN,NaN,NaN,NaN,359
1,Salvation Army,NaN,NaN,NaN,1,NaN,unknown,NaN,NaN,unknown,...,NaN,NaN,NaN,NaN,https://www.teiss.co.uk/salvation-army-ransomw...,NaN,NaN,NaN,NaN,358
2,Grupo Fleury,NaN,Brazil's largest diagnostic company,healthcare,10,686.0,unknown,NaN,NaN,unknown,...,NaN,NaN,NaN,NaN,https://www.bleepingcomputer.com/news/security...,NaN,NaN,NaN,NaN,357


## 4. Función genérica de limpieza

A continuación definimos `clean_generic(df, key_cols, num_keywords)` que aplica pasos básicos de limpieza a cualquier DataFrame:

1. Normaliza nombres de columna: todo a minúsculas, espacios y caracteres especiales → `_`.
2. Reemplaza valores “unknown” o cadenas vacías por `NaN`.
3. Convierte a numérico todas las columnas cuyo nombre contiene alguna de las palabras clave en `num_keywords`.
4. Elimina filas con `NaN` en las columnas listadas en `key_cols`.
5. Devuelve el DataFrame limpio y normalizado.

In [23]:
def clean_generic(df: pd.DataFrame,
                  key_cols: list[str],
                  num_keywords: list[str]) -> pd.DataFrame:
    """
    Limpieza básica:
      - Normaliza nombres de columnas.
      - Reemplaza 'unknown' y '' por NaN.
      - Convierte a numérico columnas con keywords.
      - Elimina filas con NaN en key_cols.
    """
    df = df.copy()
    # 1. Normalizar nombres
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r'[^0-9a-z_]', '_', regex=True)
    )
    # 2. Valores inválidos
    df.replace(['unknown', ''], np.nan, inplace=True)
    # 3. Conversión numérica
    for col in df.columns:
        if any(kw in col for kw in num_keywords):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # 4. Eliminar filas con NaN en columnas clave
    df.dropna(subset=key_cols, inplace=True)
    return df


#### Probamos `clean_generic` en cada DataFrame con sus columnas clave y palabras clave correspondientes.

In [24]:
# 6.1 IoT-Malware
df_iot_gen = clean_generic(
    df_iot,
    key_cols=['uid', 'service', 'duration'],
    num_keywords=['duration', 'bytes']
)

# 6.2 NetCrime
df_net_gen = clean_generic(
    df_net,
    key_cols=['country'],
    num_keywords=['complaints', 'losses']
)

# 6.3 RansomAttacks
df_ransom_gen = clean_generic(
    df_ransom,
    key_cols=['target', 'year', 'month'],
    num_keywords=['revenue', 'cost', 'ransom']
)

# Ver formas resultantes
print("IoT gen:", df_iot_gen.shape)
print("Net gen:", df_net_gen.shape)
print("Ransom gen:", df_ransom_gen.shape)

IoT gen: (156, 26)
Net gen: (116, 16)
Ransom gen: (353, 32)


## 5. Limpieza y transformación manual

##### Aunque la función genérica ayuda a un primer filtrado, no cubre transformaciones específicas (p. ej. parseo de timestamps, creación de indicadores reconstrucción de fechas). A continuación aplicamos limpieza manual para cada caso:
---
### 5.1 df_iot  (IoT-Malware)

**Columnas clave**:
`ts`, `uid`, `proto`, `service`, `duration`, `orig_bytes`, `resp_bytes`, `conn_state`, `label`, `detailed_label`

**Pasos**:
1. Normalizar nombres (`.` → `_`, minúsculas).
2. Convertir `duration`, `orig_bytes`, `resp_bytes` a numérico.
3. Parsear `ts` a datetime.
4. Rellenar/consolidar `conn_state` y `service` en minúsculas.
5. Generar indicadores:
   - `is_http` (service == "http")
   - `is_long`  (duration > 1 s)
   - `byte_ratio` = resp_bytes / orig_bytes

In [13]:
# 5.1.1 Normalizar nombres
df_iot.columns = (
    df_iot.columns
      .str.strip()
      .str.lower()
      .str.replace(r'[^0-9a-z_]', '_', regex=True)
)

# 5.1.2 Tipos numéricos
for col in ['duration', 'orig_bytes', 'resp_bytes']:
    df_iot[col] = pd.to_numeric(df_iot[col], errors='coerce')

# 5.1.3 Timestamp a datetime
df_iot['ts'] = pd.to_datetime(df_iot['ts'], unit='s', errors='coerce')

# 5.1.4 Normalizar texto
df_iot['service'] = df_iot['service'].str.lower().fillna('unknown')
df_iot['conn_state'] = df_iot['conn_state'].str.upper().fillna('NA')

# 5.1.5 Indicadores
df_iot['is_http']     = df_iot['service'] == 'http'
df_iot['is_long']     = df_iot['duration'] > 1.0
df_iot['byte_ratio']  = df_iot['resp_bytes'] / df_iot['orig_bytes']

df_iot.head(3)


,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,...,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed_label,is_http,is_long,byte_ratio
0,2019-01-10 20:06:29.067208052,CzsY0D4B96NTr8m7ld,192.168.1.199,59222,46.101.251.172,80,tcp,http,1.686784,149.0,...,122,7741,122,178102,-,Malicious C&C,FileDownload,True,True,1152.684564
1,2019-01-10 20:06:30.793723106,CGEJbl3RNkmXzmkEd4,192.168.1.199,59224,46.101.251.172,80,tcp,-,3.081233,0.0,...,3,180,0,0,-,Malicious C&C,NaN,False,True,NaN
2,2019-01-10 20:06:37.954878092,CVMYDw4wnZfqWrOfd,192.168.1.199,59224,46.101.251.172,80,tcp,-,NaN,NaN,...,1,60,0,0,-,Malicious C&C,NaN,False,False,NaN


### 4.2 df_net (NetCrime)
**Columnas clave**:
country, 2019_complaints … 2024_losses

**Pasos**:

1. Normalizar nombres (comas y mayúsculas).
2. Convertir todas las columnas de “complaints” y “losses” a numérico.
3. Calcular:
    - total_complaints (suma 2019–2024)
    - total_losses (suma 2019–2024)
    - losses_per_complaint = total_losses / total_complaints

In [14]:
# 5.2.1 Normalizar nombres
df_net.columns = (
    df_net.columns
      .str.strip()
      .str.lower()
      .str.replace(r'[^0-9a-z_]', '_', regex=True)
)

# 5.2.2 Convertir a numérico
num_cols = [c for c in df_net.columns if 'complaints' in c or 'losses' in c]
df_net[num_cols] = df_net[num_cols].apply(pd.to_numeric, errors='coerce')

# 5.2.3 Cálculos agregados
df_net['total_complaints'] = df_net[[c for c in num_cols if 'complaints' in c]].sum(axis=1)
df_net['total_losses']     = df_net[[c for c in num_cols if 'losses' in c]].sum(axis=1)
df_net['losses_per_complaint'] = df_net['total_losses'] / df_net['total_complaints']

df_net.head(3)



,country,2019_complaints,2019_losses,2020_complaints,2020_losses,2021_complaints,2021_losses,2022_complaints,2022_losses,2023_complaints,2023_losses,2024_complaints,2024_losses,total_complaints,total_losses,losses_per_complaint
0,PR,655,5929974,1338,7209755,1785,9463928,1594,17211758,1817,21018302,1974,31545772,9163,92379489,10081.795154
1,PS,1784,22483591,2890,25423219,3352,48918464,3210,57796003,3378,69292891,3811,66002407,18425,289916575,15734.956581
2,PT,1119,13870074,2020,12391290,2102,18205913,1918,30859319,2178,28700418,2209,40192274,11546,144219288,12490.844275


### 4.5 df_ransom  (RansomAttacks)
**Columnas clave**:
target, sector, organisation_size_1_5_10_25_100_300,
revenue_usd_million, cost, ransom_cost, year, month, date

Pasos:
1. Normalizar nombres (espacios → _, minúsculas).
2. Reemplazar "unknown" y cadenas vacías por NaN.
3. Convertir numéricos: organisation_size_1_5_10_25_100_300, revenue_usd_million, cost, ransom_cost, year.
4. Construir columna date a partir de year y month.
5. Generar indicadores:
    - ransom_to_revenue = ransom_cost / revenue_usd_million
    - large_org = organisation_size ≥ 100

In [18]:
# 5.3.a Detectar y convertir automáticamente columnas numéricas
num_cols = [c for c in df_ransom.columns
            if any(keyword in c for keyword in ['revenue', 'cost', 'ransom', 'year'])]
print("Columnas numéricas detectadas:", num_cols)

for c in num_cols:
    df_ransom[c] = pd.to_numeric(df_ransom[c], errors='coerce')

# 5.3.b Construir columna 'date'
df_ransom['month'] = df_ransom['month'].str.capitalize()
df_ransom['date'] = pd.to_datetime(
    df_ransom['month'] + ' ' + df_ransom['year'].astype(int).astype(str),
    format='%b %Y',
    errors='coerce'
)

# 5.3.c Generar indicadores
df_ransom['ransom_to_revenue'] = (
    df_ransom['ransom_cost'] / df_ransom['revenue__usd_million']
)
df_ransom['large_org'] = (
    df_ransom['organisation_size_1_5_10_25_100_300'] >= 100
)

# Mostrar primeras filas para verificar
df_ransom[['target','sector','revenue__usd_million','ransom_cost','date',
           'ransom_to_revenue','large_org']].head(5)



Columnas numéricas detectadas: ['revenue__usd_million', 'cost', 'ransom_cost', 'ransom_paid', 'year_code', 'year', 'ransomware', 'revenue_as_of']


,target,sector,revenue__usd_million,ransom_cost,date,ransom_to_revenue,large_org
0,Kaseya,tech,300.0,70.0,2021-07-01,0.233333,True
1,Salvation Army,NaN,NaN,NaN,2021-06-01,NaN,False
2,Grupo Fleury,healthcare,686.0,NaN,2021-06-01,NaN,False
3,City of Liege,government,NaN,NaN,2021-06-01,NaN,False
4,Lucky Star Casino,NaN,9.0,NaN,2021-06-01,NaN,False


## 6. Expansión de DataFrames y generación de IDs

A continuación insertamos un `id` secuencial único en cada DataFrame ya transformado, en preparación para su carga o análisis:


In [19]:
def add_sequential_id(df: pd.DataFrame, start: int = 1) -> pd.DataFrame:
    df = df.copy()
    df.insert(0, 'id', range(start, start + len(df)))
    return df

### 6.1 Aplicar ID a cada DataFrame

In [20]:
df_iot_final    = add_sequential_id(df_iot)
df_net_final    = add_sequential_id(df_net)
df_ransom_final = add_sequential_id(df_ransom)

## 6.2 Comprobar formas resultantes

In [21]:
print("IoT-Malware final:", df_iot_final.shape)
print("NetCrime final:  ", df_net_final.shape)
print("RansomAttacks final:", df_ransom_final.shape)

IoT-Malware final: (237, 27)
NetCrime final:   (117, 17)
RansomAttacks final: (358, 33)


## 7. Verificación final de contenidos
Validamos que cada DataFrame contiene la columna id y muestra correctamente sus primeros registros:

In [22]:
# 7.1 Primeras filas de df_iot_final
print("### IoT-Malware Final")
display(df_iot_final.head(3))

# 7.2 Primeras filas de df_net_final
print("### NetCrime Final")
display(df_net_final.head(3))

# 7.3 Primeras filas de df_ransom_final
print("### RansomAttacks Final")
display(df_ransom_final.head(3))


### IoT-Malware Final


,id,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,...,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed_label,is_http,is_long,byte_ratio
0,1,2019-01-10 20:06:29.067208052,CzsY0D4B96NTr8m7ld,192.168.1.199,59222,46.101.251.172,80,tcp,http,1.686784,...,122,7741,122,178102,-,Malicious C&C,FileDownload,True,True,1152.684564
1,2,2019-01-10 20:06:30.793723106,CGEJbl3RNkmXzmkEd4,192.168.1.199,59224,46.101.251.172,80,tcp,-,3.081233,...,3,180,0,0,-,Malicious C&C,NaN,False,True,NaN
2,3,2019-01-10 20:06:37.954878092,CVMYDw4wnZfqWrOfd,192.168.1.199,59224,46.101.251.172,80,tcp,-,NaN,...,1,60,0,0,-,Malicious C&C,NaN,False,False,NaN


### NetCrime Final


,id,country,2019_complaints,2019_losses,2020_complaints,2020_losses,2021_complaints,2021_losses,2022_complaints,2022_losses,2023_complaints,2023_losses,2024_complaints,2024_losses,total_complaints,total_losses,losses_per_complaint
0,1,PR,655,5929974,1338,7209755,1785,9463928,1594,17211758,1817,21018302,1974,31545772,9163,92379489,10081.795154
1,2,PS,1784,22483591,2890,25423219,3352,48918464,3210,57796003,3378,69292891,3811,66002407,18425,289916575,15734.956581
2,3,PT,1119,13870074,2020,12391290,2102,18205913,1918,30859319,2178,28700418,2209,40192274,11546,144219288,12490.844275


### RansomAttacks Final


,id,target,aka,description,sector,organisation_size_1_5_10_25_100_300,revenue__usd_million,cost,ransom_cost,data_note,...,source_name,url,url_2,url_3,url_4,url_5,_id,date,ransom_to_revenue,large_org
0,1,Kaseya,NaN,NaN,tech,300,300.0,70.0,70.0,NaN,...,NaN,https://techcrunch.com/2021/07/05/kaseya-hack-...,NaN,NaN,NaN,NaN,359,2021-07-01,0.233333,True
1,2,Salvation Army,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,https://www.teiss.co.uk/salvation-army-ransomw...,NaN,NaN,NaN,NaN,358,2021-06-01,NaN,False
2,3,Grupo Fleury,NaN,Brazil's largest diagnostic company,healthcare,10,686.0,NaN,NaN,NaN,...,NaN,https://www.bleepingcomputer.com/news/security...,NaN,NaN,NaN,NaN,357,2021-06-01,NaN,False
